In [1]:
import csv
from textacy import preprocess
import ftfy
import codecs

In [2]:
file_reader = csv.reader(open("/home/SMRT-labeled/Book1.csv", 'rb'))

title = file_reader.next()
#lines = [fl for fl in file_reader]

In [3]:
print(title)

['Category', 'Topic', 'Tonality', 'Summary']


In [4]:
Category = []
Topic = []
Tonality = []
Summary = []

for [cat, top, ton, summ] in file_reader:
    Category.append(cat)
    Topic.append(top)
    Tonality.append(ton)
    Summary.append(summ)

In [5]:
def cleanSummary(summ_i):
    preprocessed_line = preprocess.remove_accents(summ_i.decode('unicode-escape'), method=u'ascii')
    preprocessed_line = preprocess.normalize_whitespace(preprocessed_line)
    preprocessed_line = preprocess.replace_urls(preprocessed_line)
    preprocessed_line = preprocessed_line.replace("\n", '')
    
    return preprocessed_line

Summary = map(cleanSummary, Summary)
print(len(Summary))

4175


In [6]:
#remove duplicated summary and get index filter
Summary_set = list(set(Summary))
index_filter = [Summary.index(x) for x in Summary_set]
print(len(Summary_set))

1766


In [7]:
def cleanMeta(meta_i):
    preprocessed_line = preprocess.fix_bad_unicode(meta_i.decode('unicode-escape'), normalization=u'NFKD')
    preprocessed_line = preprocess.normalize_whitespace(preprocessed_line)
    
    if(len(preprocessed_line) == 0):
        preprocessed_line = u'Unknown'
    
    return preprocessed_line.lower()

Category = map(cleanMeta, Category)
Topic = map(cleanMeta, Topic)
Tonality = map(cleanMeta, Tonality)

Category_set = [Category[i] for i in index_filter]
Topic_set = [Topic[i] for i in index_filter]
Tonality_set = [Tonality[i] for i in index_filter]

In [8]:
#check category before putting it as meta
print(set(Category_set))
print(len(Category_set))

print(len(list(set(Category_set))), len(list(set(Category))))

set([u'taxi', u'overseas news', u'public transport industry', u'forum', u'smrt corporate', u'competitors', u'smrt buses', u'unknown', u'smrt taxis', u'smrt trains'])
1766
(10, 10)


In [9]:
#check Topic before putting it as meta
for i in list(set(Topic_set)):
    print i
print(len(list(set(Topic_set))), len(list(set(Topic))))

corporate & financials
service excellence
facilities
crisis / delays & distributions / accidents / safety
financials
labour & union
unknown
labour
general / others
repair/maintenance
facilities & services
fines
csr
customer satisfaction
corporate social responsibility
customer service
commuter behaviour
rail & engineering
regulations
service announcements
fares
safety/accident
regulations & ops
delay/disruption
(24, 24)


In [10]:
#check Tonality before putting it as meta
print(set(Tonality_set))
print(len(Tonality_set))

set([u'unknown', u'neutral', u'negative', u'positive'])
1766


In [11]:
#summary to corpus
from textacy import corpus

metadata = map(lambda x:{'Category':x[0], 'Topic':x[1], 'Tonality':x[2]}, zip(Category_set, Topic_set, Tonality_set))
smrt_summary = corpus.Corpus(u'en', texts = Summary_set, metadatas = metadata)

#save corpus to disk for future use
#smrt_summary.save('/home/SMRT-labeled/', name='summary', compression='gzip')
#failed to load!!!

In [12]:
from textacy import fileio

#fileio.write_spacy_docs([d.spacy_doc for d in smrt_summary.docs], '/home/SMRT-labeled/summary-docs')

fileio.write_file_lines([d.spacy_doc.text for d in smrt_summary.docs], '/home/SMRT-labeled/summary-docs')
fileio.write_json(metadata, '/home/SMRT-labeled/summary-meta')